In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/jds/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import sklearn

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
df = df[df.columns.drop(list(df.filter(regex='err')))]
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,...,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,...,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,...,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,...,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,...,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


# Select your features (columns)

In [9]:
y = df['koi_disposition']
X = df.drop("koi_disposition", axis=1)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [10]:
y = df['koi_disposition']
print(X.shape, y.shape)

(6991, 20) (6991,)


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [15]:
labelencoder.fit(y_train)

LabelEncoder()

In [16]:
y_train_encoded = labelencoder.transform(y_train)

In [17]:
y_test_encoded= labelencoder.transform(y_test)

In [18]:
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [19]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler 
X_scaler = MinMaxScaler().fit(X_train)

In [20]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation="relu", input_dim=20))
model.add(Dense(units=100, activation="relu"))
model.add(Dense(units=3, activation="softmax"))

In [23]:
X_train_scaled.shape

(5243, 20)

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 12,503
Trainable params: 12,503
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
164/164 - 1s - loss: 0.5335 - accuracy: 0.7080
Epoch 2/60
164/164 - 0s - loss: 0.3900 - accuracy: 0.7896
Epoch 3/60
164/164 - 0s - loss: 0.3802 - accuracy: 0.7971
Epoch 4/60
164/164 - 0s - loss: 0.3767 - accuracy: 0.7923
Epoch 5/60
164/164 - 0s - loss: 0.3731 - accuracy: 0.7965
Epoch 6/60
164/164 - 0s - loss: 0.3709 - accuracy: 0.7992
Epoch 7/60
164/164 - 0s - loss: 0.3694 - accuracy: 0.8020
Epoch 8/60
164/164 - 0s - loss: 0.3719 - accuracy: 0.8022
Epoch 9/60
164/164 - 0s - loss: 0.3648 - accuracy: 0.8056
Epoch 10/60
164/164 - 0s - loss: 0.3620 - accuracy: 0.8053
Epoch 11/60
164/164 - 0s - loss: 0.3600 - accuracy: 0.8037
Epoch 12/60
164/164 - 0s - loss: 0.3624 - accuracy: 0.8093
Epoch 13/60
164/164 - 0s - loss: 0.3597 - accuracy: 0.8135
Epoch 14/60
164/164 - 0s - loss: 0.3584 - accuracy: 0.8106
Epoch 15/60
164/164 - 0s - loss: 0.3573 - accuracy: 0.8119
Epoch 16/60
164/164 - 0s - loss: 0.3540 - accuracy: 0.8121
Epoch 17/60
164/164 - 0s - loss: 0.3577 - accuracy: 0.8127
Epoch 

In [27]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.3141 - accuracy: 0.8593
Normal Neural Network - Loss: 0.3141436278820038, Accuracy: 0.8592677116394043


# Train the Model



In [28]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.3141 - accuracy: 0.8593
Normal Neural Network - Loss: 0.3141436278820038, Accuracy: 0.8592677116394043


# Save the Model

In [30]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'jds_1.sav'
joblib.dump(model, filename)

TypeError: cannot pickle 'weakref' object